In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import requests
import time
from tqdm import tqdm
import os
import re
import datetime

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException # 검색 결과가 없는 경우 예외 처리
from bs4 import BeautifulSoup

In [33]:
# 수집 날짜 지정(전일자)
before_one_day = datetime.datetime.now() - datetime.timedelta(days=1)
D, d = before_one_day.strftime("%Y.%m.%d"), before_one_day.strftime("%Y%m%d")
db_date = d[2:]

In [46]:
# conn = db_connection('')
# news_link = pd.read_sql(f"select * from db_name where is_status = 1 and text IS NULL;", conn)
# conn.close()

In [ ]:
news_link = pd.read_xlsx(f'naver_news_{d}.xlsx')

In [47]:
news_link = news_link[(news_link['is_status']==1)&(news_link['text'].isnull()==True)]

print(len(news_link))
news_link.head()

2889


,id,ck_idx,news,title,href,is_status,text,daum,write_date,regist_date
0,6013,46,뉴스엔,"‘FNC 7인조 신인’ 앰퍼샌드원, 11월 15일 데뷔 확정 [공식]",https://n.news.naver.com/mnews/article/609/000...,1,None,None,None,2023-10-26 00:20:54
1,6016,46,엑스포츠뉴스,"임성언, 오에스알엔터와 전속계약 '다방면 활동 예고'",https://n.news.naver.com/mnews/article/311/000...,1,None,None,None,2023-10-26 00:20:54
2,6017,46,엑스포츠뉴스,"에이티즈 성화, 조모상으로 일정 불참 ""발인 후 복귀"" [공식입장]",https://n.news.naver.com/mnews/article/311/000...,1,None,None,None,2023-10-26 00:20:54
3,6020,46,엑스포츠뉴스,'전역' 이대열 팀 활동 복귀→골든차일드 컴백 포토 공개,https://n.news.naver.com/mnews/article/311/000...,1,None,None,None,2023-10-26 00:20:54
4,6021,46,스포티비뉴스,"'FNC 새 보이그룹' 앰퍼샌드원, 11월 15일 데뷔 확정",https://n.news.naver.com/mnews/article/477/000...,1,None,None,None,2023-10-26 00:20:54


---

In [48]:
# 뉴스 내용 담을 DataFrame
news_text = pd.DataFrame(columns = ['title', 'text', 'write_date'])

drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for i in tqdm(range(len(news_link))):
    drv.get(news_link['href'][i])
    time.sleep(1.5)
    soup = BeautifulSoup(drv.page_source, 'html.parser')
    
    try:
        title = soup.find('h2', {'class':'end_tit'}).get_text().replace('\n', '').replace('\t', '')
    except:
        title = soup.find('h2', {'id':'title_area'}).get_text()
    
    try:
        date = soup.find('div', {'class':'article_info'}).find('span', {'class':'author'}).get_text()
        date = date.split('기사입력')[-1].split(' ')[0][:-1].replace('.', '-')
    except:
        date = soup.find('span', {'class':'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'}).get_text()
        date = date.split(' ')[0][:-1].replace('.', '-')
    
    try:
        try:
            del_w = soup.find('div', {'id':'articeBody'}).find('em', {'class':'img_desc'}).get_text()
            text = soup.find('div', {'id':'articeBody'}).get_text()
            text = text.replace('\n', ' ').split(del_w)[-1].replace('\'', '').replace('\t', '').replace('"', '').replace.strip()
            
            if len(soup.find('div', {'id':'articeBody'}).get_text().replace('\n', ' ').split(del_w))>2:
                text = soup.find('div', {'id':'articeBody'}).get_text()
                text = text.replace('\n', ' ').replace('\'', '').replace('\t', '').replace('"', '').strip()      
        except:
            text = soup.find('div', {'id':'articeBody'}).get_text()
            text = text.replace('\n', ' ').replace('\'', '').replace('\t', '').replace('"', '').strip()
    except:
        try:
            del_w = soup.find('div', {'id':'newsct_article'}).find('em', {'class':'img_desc'}).get_text()
            text = soup.find('article', {'id':'dic_area'}).get_text()
            text = text.replace('\n', '').split(del_w)[-1].replace('\xa0', '').replace('\t', '').replace('\'', '').replace('"', '').strip()
            
            if len(soup.find('article', {'id':'dic_area'}).get_text().replace('\n', ' ').split(del_w))>2:
                text = soup.find('article', {'id':'dic_area'}).get_text()
                text = text.replace('\n', '').replace('\xa0', '').replace('\t', '').replace('\'', '').replace('"', '').strip()
        except:
            text = soup.find('article', {'id':'dic_area'}).get_text()
            text = text.replace('\n', '').replace('\xa0', '').replace('\t', '').replace('\'', '').replace('"', '').strip()
        
    data = pd.DataFrame({'id':news_link['id'][i],'title':[title], 'write_date':[date], 'text':[text]})
    news_text = pd.concat([news_text, data], ignore_index=True)

drv.close()

100%|█████████████████████████████████████████| 489/489 [19:13<00:00,  2.36s/it]


In [49]:
news_text

,title,text,write_date,id
0,"평창군-서울대 그린바이오과학기술연구원, 대화면 도시재생 '맞손'",【평창】대화면 도시재생사업 활성화를 위해 평창군과 서울대 그린바이로과학기술연구원이 ...,2023-10-25,15311.0
1,'무안 먹거리 통합플랫폼' 전남형 전략사업으로…사업비 100억 확보,(무안=뉴스1) 김태성 기자 = 전남 무안군은 2024년 전남형 지역성장 전략사업에...,2023-10-25,15346.0
2,강원특별자치도 백년‧유망중소기업’ 20개사 선정,"강원특별자치도는 최근 도 중소기업지원기관협의회 심의를 거쳐 백년기업 5개사, 유망중...",2023-10-25,15347.0
3,"[전남소식] 장성 천진암, 정관스님과 함께하는 사찰음식 등",[장성=뉴시스]김혜인 기자 = 전남 장성 천진암은 오는 26일부터 이틀간 정관스님과...,2023-10-25,15358.0
4,막내아들의 '광폭 행보'…한화그룹서 존재감 키우는 김동선,김동선 한화갤러리아 전략본부장이 한화갤러리아 지분을 잇달아 매입하고 있다. 김 본부...,2023-10-25,15364.0
...,...,...,...,...
484,국민의힘서 뜨는 '험지 출마론'에 당 내부 '갑론을박',[데일리안 = 김민석 기자] 국민의힘 내부에서 중진 의원들을 향한 험지 출마론 요구...,2023-10-25,17604.0
485,"하나투어, 2023 미디어데이 개최…송미선 대표 ""향후 3년간 시장점유율 35% 목표""",송미선 하나투어 대표가 향후 3년간 시장점유율을 35%까지 끌어올리겠다고 밝혔다. ...,2023-10-25,17608.0
486,스타트업 아이디어 탈취 의혹 논란 네이버…최수연 대표 국감 증인 채택,"네이버가 스타트업 아이디어를 탈취했다는 의혹이 제기됐다. 네이버는 즉각 반발했으나,...",2023-10-25,17610.0
487,"경북 영주, '관광 핫플'로 뜬다…지방소멸 해결사로 나선 스타트업",국내 스타트업 액셀러레이터 블루포인트파트너스는 지방소멸 극복 아이디어를 가진 스타트...,2023-10-25,17617.0


In [50]:
news_text['text'] = [news_text['text'][i].replace('이미지 원본보기', '').strip() for i in range(len(news_text))]

In [ ]:
# 저장(덮어쓰기)
news_text.to_xlsx(f'naver_news_{d}.xlsx')

In [51]:
# for i in tqdm(range(len(news_text))):
#     conn = db_connection('')
#     cur = conn.cursor()

#     cur.execute(
#         f'update db_name set text="{news_text["text"][i]}", write_date="{news_text["write_date"][i]}" where id = "{news_text["id"][i]}"'
#     )
#     conn.commit()

#     cur.close()
#     conn.close()

100%|█████████████████████████████████████████| 489/489 [02:35<00:00,  3.15it/s]
